# Let's Solve the Content Generation Problem with OpenAI

1. **Pass the Main Plant to an API for Keyword Extraction**  
   - Google Keyword API  
   - Semrush API  
   - Ahrefs API  

2. **Pass Relevant Keywords to OpenAI for Content Generation**  
   - Divide content generation into sections  
   - Filter keywords for each section  

3. **Ensure a Well-Defined Layout**  
   - Structure the content for SEO optimization  

4. **Store the Generated Content in BigQuery**  
   - Follow the usual workflow for storage  

In [1]:
from openai import OpenAI
import re
import json
from google.ads.googleads.client import GoogleAdsClient
import pandas as pd
import requests
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Plant API - Image

In [45]:
def get_species_list(api_key, page=1, query=None):
    base_url = "https://perenual.com/api/species-list"
    params = {
        "key": api_key,
        "page": page
    }
    
    # Add the query parameter if provided
    if query:
        params["q"] = query
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for HTTP errors
        return response.json()  # Return the response as JSON
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

In [47]:
# Example usage:
api_key = ""  # Replace with your actual API key
page_number = 1  # Example page number
search_query = "Pittosporum Tobira"  # Example query (optional)

species_data = get_species_list(api_key, page=page_number, query=search_query)
display(species_data) 

{'data': [{'id': 6172,
   'common_name': 'Japanese pittosporum',
   'scientific_name': ['Pittosporum tobira'],
   'other_name': [],
   'cycle': "Upgrade Plans To Premium/Supreme - https://perenual.com/subscription-api-pricing. I'm sorry",
   'watering': "Upgrade Plans To Premium/Supreme - https://perenual.com/subscription-api-pricing. I'm sorry",
   'sunlight': "Upgrade Plans To Premium/Supreme - https://perenual.com/subscription-api-pricing. I'm sorry",
   'default_image': {'license': 451,
    'license_name': 'CC0 1.0 Universal (CC0 1.0) Public Domain Dedication',
    'license_url': 'https://creativecommons.org/publicdomain/zero/1.0/',
    'original_url': 'https://perenual.com/storage/image/upgrade_access.jpg',
    'regular_url': 'https://perenual.com/storage/image/upgrade_access.jpg',
    'medium_url': 'https://perenual.com/storage/image/upgrade_access.jpg',
    'small_url': 'https://perenual.com/storage/image/upgrade_access.jpg',
    'thumbnail': 'https://perenual.com/storage/image/

# Plant API - Zyla

In [48]:
def lookup_by_taxonomic_name(api_key, taxonomic_name):
    url = "https://zylalabs.com/api/1852/plants+data+api/1515/lookup+by+taxonomic+name"
    headers = {
        "Authorization": f"Bearer {api_key}"  # Include the Bearer token
    }
    params = {
        "SN": taxonomic_name  # Taxonomic name (Scientific Name)
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Check for HTTP errors
        return response.json()  # Return the response as JSON
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

In [50]:
# Example usage:
api_key = ""  # Replace with your actual API key
taxonomic_name = "Pittosporum Tobira"  # Example taxonomic name

plant_data = lookup_by_taxonomic_name(api_key, taxonomic_name)
display(plant_data)

[{'_id': '63ef4ed5476230641c4c6b63',
  'UID': 24069,
  'pid': 25945,
  'Genus': 'Pittosporum',
  'Species': 'NA',
  'ScientificName': 'Pittosporum tobira',
  'CommonName': 'Japanese cheesewood',
  'Category': 'Dicot',
  'Family': 'Pittosporaceae',
  'FamilySymbol': 'PITTOS',
  'FamilyCommonName': 'Pittosporum family',
  'xOrder': 'Rosales',
  'SubClass': 'Rosidae',
  'Class': 'Magnoliopsida',
  'Division': 'Magnoliophyta',
  'SuperDivision': 'Spermatophyta',
  'SubKingdom': 'Tracheobionta',
  'Kingdom': 'Plantae',
  'ITIS_TSN': '24067',
  'Duration': 'Perennial',
  'GrowthHabit': 'Tree, Shrub',
  'ActiveGrowthPeriod': 'Spring, Summer, Fall',
  'Bloat': 'None',
  'C2N_Ratio': 'High',
  'CoppicePotential': 'No',
  'FallConspicuous': 'No',
  'FireResistance': 'Yes',
  'FlowerColor': 'White',
  'FlowerConspicuous': 'Yes',
  'FoliageColor': 'White-Gray',
  'FoliagePorositySummer': 'Dense',
  'FoliagePorosityWinter': 'Dense',
  'FoliageTexture': 'Coarse',
  'FruitColor': 'Orange',
  'FruitCo

In [53]:
def extract_relevant_info(plant_data):
    # Assuming `plant_data` is a list with at least one dictionary item
    plant = plant_data[0]
    
    # Extract the relevant fields
    common_name = f"{plant.get('CommonName', 'Unknown')}"
    scientific_name = f"{plant.get('ScientificName', 'Unknown')}"
    family = f"{plant.get('Family', 'Unknown')}"
    typical_height = f"{plant.get('MatureHeight', 'Unknown')}"
    min_temp_tolerance = f"Can tolerate temperatures down to {plant.get('TemperatureMinimum', 'Unknown')}°C"
    climate = "Suitable for coastal, temperate, and subtropical regions"  # Based on common knowledge of Pittosporum tobira
    light_conditions = f"Shade tolerance: {plant.get('ShadeTolerance', 'Unknown')}"
    growth_habit = f"{plant.get('GrowthForm', 'Unknown')} with {plant.get('FlowerColor', 'Unknown')} flowers"
    growth_conditions = f"Well-drained soil, tolerates {', '.join([soil for soil, adapted in {'coarse': plant.get('AdaptedCoarseSoils', 'No'), 'medium': plant.get('AdaptedMediumSoils', 'No'), 'fine': plant.get('AdaptedFineSoils', 'No')}.items() if adapted == 'Yes'])}, requires {plant.get('MoistureUse', 'Unknown')} watering, prefers pH between {plant.get('pH_Minimum', 'Unknown')} and {plant.get('pH_Maximum', 'Unknown')}"

    # Creating a dictionary to store the extracted data
    extracted_info = {
        "Common Name" : common_name,
        "Scientific Name" : scientific_name,
        "Family" : family,
        "Typical height": typical_height,
        "Minimum temperature tolerance": min_temp_tolerance,
        "Climate": climate,
        "Light conditions": light_conditions,
        "Growth habit": growth_habit,
        "Growth conditions": growth_conditions
    }

    return extracted_info

In [54]:
zyla_info_for_info_section = extract_relevant_info(plant_data=plant_data)

## API for Keyword Extraction

In [9]:
client = GoogleAdsClient.load_from_storage("cred.yaml")
client.login_customer_id = "3494351133"  # Manager account ID

keyword_plan_idea_service = client.get_service("KeywordPlanIdeaService")
request = client.get_type("GenerateKeywordIdeasRequest")
request.customer_id = "3494351133"  # Test account ID
request.keyword_plan_network = client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH

# Change geo target constant to Australia
request.geo_target_constants.append("geoTargetConstants/2036")  # Australia

request.language = "languageConstants/1000"  # English language constant

In [10]:
# Set multiple seed keywords
seed_keywords = ["Pittosporum Tobira", "what is Pittosporum Tobira?", "where to buy Pittosporum Tobira?", "what's the common name of Miss Muffet?"]

# Add each keyword to the request
for keyword in seed_keywords:
    request.keyword_seed.keywords.append(keyword)

# Make the API call
response = keyword_plan_idea_service.generate_keyword_ideas(request=request)

# Print the results for each keyword idea
for idea in response.results:
    print(f"Keyword: {idea.text}, Avg. Monthly Searches: {idea.keyword_idea_metrics.avg_monthly_searches}")


Keyword: pittosporum tobira, Avg. Monthly Searches: 590
Keyword: pittosporum tobira nanum, Avg. Monthly Searches: 30
Keyword: japanese pittosporum, Avg. Monthly Searches: 590
Keyword: japanese cheesewood, Avg. Monthly Searches: 210
Keyword: japanese mock orange, Avg. Monthly Searches: 40
Keyword: pittosporum nanum, Avg. Monthly Searches: 10
Keyword: japanese pittosporum for sale, Avg. Monthly Searches: 10
Keyword: acheter pittosporum tobira, Avg. Monthly Searches: 0
Keyword: azarero pittosporum tobira, Avg. Monthly Searches: 0
Keyword: cheesewood plant, Avg. Monthly Searches: 20
Keyword: haie de pittosporum tobira, Avg. Monthly Searches: 0
Keyword: haie pittosporum tobira, Avg. Monthly Searches: 0
Keyword: japanese cheesewood for sale, Avg. Monthly Searches: 10
Keyword: japanese cheesewood plant, Avg. Monthly Searches: 10
Keyword: japanese cheesewood tree, Avg. Monthly Searches: 10
Keyword: japanese mock orange pittosporum tobira, Avg. Monthly Searches: 10
Keyword: japanese mock orange

In [11]:
type(response)

google.ads.googleads.v17.services.services.keyword_plan_idea_service.pagers.GenerateKeywordIdeasPager

In [12]:
# Prepare data for the DataFrame
data = []
for idea in response.results:
    # Extract the keyword text, average monthly searches, and competition index
    keyword = idea.text
    avg_monthly_searches = idea.keyword_idea_metrics.avg_monthly_searches
    competition_index = idea.keyword_idea_metrics.competition_index
    
    # Append to the data list
    data.append({
        "text": keyword,
        "avg_monthly_searches": avg_monthly_searches,
        "competition_index": competition_index
    })

In [13]:
# Create a DataFrame from the collected data
df = pd.DataFrame(data)

In [14]:
df.head()

,text,avg_monthly_searches,competition_index
0,pittosporum tobira,590,27
1,pittosporum tobira nanum,30,11
2,japanese pittosporum,590,27
3,japanese cheesewood,210,9
4,japanese mock orange,40,15


In [23]:
df_sorted_comp = df.sort_values(by="competition_index", ascending=True)

# Display the sorted DataFrame
df_sorted_comp.head(20)

,text,avg_monthly_searches,competition_index
40,pittosporum scented flowers,10,0
57,pittosporum tobira precio,10,0
24,klebsame pittosporum,0,0
25,klebsame pittosporum tobira,0,0
26,le pittosporum tobira,0,0
27,miss muffet pittosporum tobira,10,0
54,pittosporum tobira nanum pruning,10,0
53,pittosporum tobira nain,0,0
31,pianta pittosporum tobira,0,0
32,pitosporo pittosporum tobira,10,0


In [15]:
df_sorted_search = df.sort_values(by='avg_monthly_searches', ascending=False)
df_sorted_search.head(20)

,text,avg_monthly_searches,competition_index
0,pittosporum tobira,590,27
20,japanese pittosporum pittosporum tobira,590,27
2,japanese pittosporum,590,27
36,pittosporum pittosporum tobira,590,27
28,p tobira,590,27
3,japanese cheesewood,210,9
16,japanese mock orange plant,40,15
4,japanese mock orange,40,15
1,pittosporum tobira nanum,30,11
9,cheesewood plant,20,2


In [17]:
# Sort the DataFrame by competition_index first and then by avg_monthly_searches in ascending order
df_sorted = df.sort_values(by=["competition_index", "avg_monthly_searches"], ascending=[True, False])

# Display the sorted DataFrame
df_sorted

,text,avg_monthly_searches,competition_index
6,japanese pittosporum for sale,10,0
13,japanese cheesewood plant,10,0
14,japanese cheesewood tree,10,0
17,japanese mock orange tree,10,0
19,japanese pittosporum indoor,10,0
31,pitosporo tobira,10,0
33,pittosporum fragrant flowers,10,0
35,pittosporum mock orange tree,10,0
38,pittosporum tenuifolium nanum,10,0
43,pittosporum tobira for sale,10,0


In [26]:
df_sorted_search[:30]

,text,avg_monthly_searches,competition_index
3,japanese pittosporum,3600,85
30,p tobira,3600,85
21,japanese pittosporum pittosporum tobira,3600,85
1,pittosporum tobira,3600,85
39,pittosporum pittosporum tobira,3600,85
4,japanese cheesewood,2900,27
28,mock orange pittosporum,390,100
5,japanese mock orange,320,94
17,japanese mock orange plant,320,94
23,japanese pittosporum variegated,210,100


In [27]:
# Get the top 10 keywords from the sorted DataFrame
selected_keywords = df_sorted.head(30)["text"].tolist()

# Display the list of keywords
selected_keywords

['azarero pittosporum tobira',
 'miss muffet pittosporum tobira',
 'pitosporo pittosporum tobira',
 'pitosporo tobira',
 'pitto tobira',
 'pittosporum miss muffet height',
 'pittosporum mock orange tree',
 'pittosporum scented flowers',
 'pittosporum tenuifolium nanum',
 'pittosporum tobira enano',
 'pittosporum tobira growth rate',
 'pittosporum tobira nanum pruning',
 'pittosporum tobira precio',
 'pittosporum tobira thunb wt aiton',
 'plante pittosporum',
 'tobira nanum',
 'acheter pittosporum tobira',
 'haie de pittosporum tobira',
 'haie pittosporum tobira',
 'klebsame pittosporum',
 'klebsame pittosporum tobira',
 'le pittosporum tobira',
 'pianta pittosporum tobira',
 'pittosporum tobira en pot',
 'pittosporum tobira in vaso',
 'pittosporum tobira nain',
 'pittosporum tobira prezzo',
 'pittosporum tobira prix',
 'pittosporum tobira siepe',
 'plante pittosporum tobira']

## Making API call to OpenAI

In [28]:
client = OpenAI(api_key='')

In [29]:
plant_template_2 = {
    "header" : """
        <header>
            <h1>Westringia Zena</h1>
            <h2><em>Westringia Fruticosa</em></h2>
        </header>

        <section id="quick-look">
            <p>The attractive Westringia Fruticosa ‘Zena’ is made more appealing by its hardiness, an attribute that typically characterises native Australian shrubs. Its ability to deal with salty winds makes it great for coastal plantings, hence, also making it suitable for coastal gardens.</p>
            <p>Being drought and frost-tolerant, basically hardy in many other locations, and possessing a sense of subtle charm and elegance, Westringia Zena is suitable for numerous garden types.</p>
            <p><strong>Synonyms:</strong> Coastal Rosemary, Native Rosemary, Australian Rosemary</p>
        </section>
    """,
    
    "introduction": """
        <section id="introduction">
            <h2>Westringia Zena for a Charming and Resilient Garden</h2>
            <p>The Westringia Fruticosa ‘Zena’ is a hardy and compact plant that’s best for various garden styles. Its compact size makes it suitable for a low hedge and smaller gardens. If you have a smaller space to allot for a garden, this Australian rosemary plant is for you.</p>
            <p>Appearance-wise, it features a dense grey-green foliage and pale mauve to white blooms, which can be quite the spectacle for most of the year. They respond beautifully to clippings and shapings, making them an excellent hedge and topiary specimen evocative of formal gardens. This plant is also well-favoured by local gardeners for its drought tolerance and minimal care requirements.</p>
            <p>Ready to transform your garden with this hardy native? Here's a comprehensive guide to help you get started.</p>
        </section>'
    """,
    
    "how-to-plant" : """
        <section id="how-to-plant">
            <h2>How to Plant Westringia Fruticosa 'Zena'?</h2>
            <p>If your garden needs another hardy plant to boost your outdoor space’s charm and resilience, Westringia Zena is worth considering. Here are essential planting tips to ensure a lovely and healthy plant:</p>
            
            <ul>
                <li>Choosing the right soil and location for planting makes a great deal of difference. When it comes to Westringia Zena, pick a sunny to partially shaded location with well-draining soil. Specifically, this plant thrives in sandy or loamy soils, common in coastal areas.</li>
                <li>After choosing the perfect spot for planting, improve soil structure by adding compost or organic matter as needed. This helps to enhance drainage and nutrient content.</li>
                <li>Once planted, water thoroughly to help settle the soil around the roots.</li>
                <li>If planting multiple Westringia plants, ensure they’re spaced about 1 to 1.5 metres apart for proper air circulation and optimal growth.</li>
                <li>Mulching around the base of the plant helps retain moisture, suppress weeds, and regulate soil temperature.</li>
            </ul>
        </section>
    """,
    
    "why-part": """
        <section id="why-part">
            <h2>Why Should You Buy Westringia Fruticosa ‘Zena’?</h2>
            <p>Westringia Fruticosa Zena offers not only aesthetic appeal but also several practical benefits, great for any of your outdoor space ideas, some of which we’ve listed below:</p>

            <ul>
                <li><strong>Attractive Foliage:</strong> Westringia fruticosa’s lush, silver-grey foliage is a great addition to any gardens, smoothly contrasting with other plants.</li>
                <li><strong>Low Maintenance:</strong> Once established, this plant doesn’t need much care and maintenance, an appealing attribute for busy and newbie gardeners alike.</li>
                <li><strong>Compact Size:</strong> Growing to around 1 metre in height and width, Zena is easily suitable for small spaces. It can also work as a neat border, hedge, or a screening plant thanks to its compact habit.</li>
                <li><strong>Drought Tolerance:</strong> Westringia Zena’s hardiness makes it well-suited for Australian gardens since it can tolerate dry and coastal conditions.</li>
                <li><strong>Frost Tolerance:</strong> Once established, Zena can also withstand light frost, making it one of the most resilient plants ever.</li>
            </ul>
        </section>
    """,
    
    "planting_guide": """
        <section id="planting-guide">
            <h2>How to Plant Westringia Fruticosa 'Zena'?</h2>
            <p>Westringia Zena is impressively hardy, but it still needs some care and attention, especially in the beginning. Once established, the occasional monitoring is still recommended to ensure your plant has no ongoing problems like pests or diseases.</p>

            <p>With these easy care steps, Westringia Zena will thrive and maintain its fancy appearance:</p>

            <ul>
                <li>Water regularly when first planted, but once established, water only during dry periods.</li>
                <li>Fertilise annually. Specifically, feed with a slow-release native plant fertiliser in spring to promote healthy growth.</li>
                <li>Light pruning after flowering will encourage thicker and new growth, as well as help retain its compact form. Prune back any leggy growth, but only as needed. <a href="your-pruning-guide-link" target="_blank">Read our guide to pruning plants to learn more.</a></li>
                <li>Although hardy and able to tolerate light frosts, you may need to protect young plants in colder climates. Mature plants, meanwhile, will also be fine with mild frost.</li>
            </ul>
        </section>
    """,
    
    "reasons_to_buy": """
        <section id="reasons-to-buy">
            <h2>Why Should You Buy Westringia Fruticosa 'Zena'?</h2>
            <p>Westringia Fruticosa Zena offers not only aesthetic appeal but also several practical benefits, great for any of your outdoor space ideas, some of which we’ve listed below:</p>

            <ul>
                <li><strong>Attractive Foliage:</strong> Westringia fruticosa’s lush, silver-grey foliage is a great addition to any gardens, smoothly contrasting with other plants.</li>
                <li><strong>Low Maintenance:</strong> Once established, this plant doesn’t need much care and maintenance, an appealing attribute for busy and newbie gardeners alike.</li>
                <li><strong>Compact Size:</strong> Growing to around 1 metre in height and width, Zena is easily suitable for small spaces. It can also work as a neat border, hedge or a screening plant thanks to its compact habit.</li>
                <li><strong>Drought Tolerance:</strong> Westringia Zena’s hardiness makes it well-suited for Australian gardens since it can tolerate dry and coastal conditions.</li>
                <li><strong>Frost Tolerance:</strong> Once established, Zena can also withstand light frost, making it one of the most resilient plants ever.</li>
            </ul>
        </section>
    """,
        
    "care_guide": """
        <section id="care-guide">
            <h2>How to Care for Westringia Fruticosa 'Zena'?</h2>
            <p>Westringia Zena is impressively hardy, but it still needs some care and attention, especially in the beginning. Once established, the occasional monitoring is still recommended to ensure your plant has no ongoing problems like pests or diseases.</p>

            <p>With these easy care steps, Westringia Zena will thrive and maintain its fancy appearance:</p>

            <ul>
                <li>Water regularly when first planted, but once established, water only during dry periods.</li>
                <li>Fertilise annually. Specifically, feed with a slow-release native plant fertiliser in spring to promote healthy growth.</li>
                <li>Light pruning after flowering will encourage thicker and new growth, as well as help retain its compact form. Prune back any leggy growth, but only as needed. <a href="your-pruning-guide-link" target="_blank">Read our guide to pruning plants to learn more.</a></li>
                <li>Although hardy and able to tolerate light frosts, you may need to protect young plants in colder climates. Mature plants, meanwhile, will also be fine with mild frost.</li>
            </ul>
        </section>
    """,

    "growing_considerations": """
        <section id="growing-considerations">
            <h2>Things to Consider When Growing Westringia Zena</h2>
            <p>Westringia Zena is incredibly hardy—being frost, salt, and wind tolerant all at the same time. This means the plant can grow and thrive anywhere. Nonetheless, there are factors that need considering for best results.</p>

            <p>First, is its suitability to very cold areas. While younger plants can tolerate low frost and mature plants can withstand mild frost, Westringia Fruticosa Zena may struggle in areas that experience very cold winters.</p>

            <p>Pruning is also required when growing this Westringia variation, even though it’s supposed to be low-maintenance. Without pruning, the plant can become unruly, losing its charming and clean aesthetic. This plant is also sensitive to poor drainage, leading to root rot if not addressed quickly.</p>

            <p>When grown in ideal conditions, the Westringia Fruticosa plant can become invasive, spreading aggressively and overshadowing other garden plants. Lastly, Westringia Fruticosa’s small blooms may not be enough for gardeners who prefer flowering plants that produce big, noticeable flowers.</p>
        </section>
    """,
    
    "pests_diseases": """
        <section id="pests-diseases">
            <h2>Treating Westringia Zena for Pests and Diseases</h2>
            <p>Westringia Zena is hardy and resistant to most pests and diseases. However, aphids and scale insects can still plague the plant. When an infestation occurs, the most common solution applied is either an insecticidal soap or neem oil.</p>

            <p>Fungal diseases can also affect Westringia Fruticosa due to poor drainage. You can easily be proactive about this issue by ensuring proper drainage and avoiding overwatering. Not sure whether you're overwatering or underwatering your plant? <a href="https://localbotanist.com.au/news/deciphering-plant-signals-the-telltale-signs-of-overwatering-vs-underwatering/">Find out using this guide.</a></p>
        </section>
    """,
    
    "purchase": """
        <section id="purchase">
            <h2>Where Can I Buy Westringia Fruticosa 'Zena'?</h2>
            <p>Looking to add this reliable Australian native to your garden? You can find Westringia Zena right here at the Local Botanist. Browse for more green additions and <a href="https://localbotanist.com.au/design-your-garden/">create your dream garden</a> with our wide selection of plants.</p>

            <p>You can also discover the garden style that’s right for you by taking our <a href="https://localbotanist.com.au/quiz/">Garden Style Quiz</a>.</p>
        </section>
    """,
    
    "companion_plants": """
        <section id="companion-plants">
            <h2>Companion Plants for Westringia Fruticosa 'Zena'</h2>
            <p>Westringia Zena fits seamlessly into a range of garden styles and pairs well with other drought-tolerant, low-maintenance plants. Some ideal companion plants include:</p>

            <ul>
                <li><strong>Coastal Rosemary:</strong> A larger cousin of Zena, it complements with similar foliage but a stronger growth habit.</li>
                <li><strong>Wynyabbie Gem:</strong> This Australian native plant easily captures attention for its charming flowers.</li>
                <li><strong>Lavandula The Princess:</strong> Bred in Australia, the deep pink flowers and grey-green foliage of The Princess pairs well with the subtle blooms of Westringia Zena.</li>
                <li><strong>No Mow Grass:</strong> These grasses add movement and texture when planted alongside Westringia. Even better, they’re also low maintenance with unique features and landscaping benefits!</li>
            </ul>
        </section>
    """
}

# Example usage:
for section_name, section_html in plant_template_2.items():
    print(f"Section: {section_name}\n{section_html}\n")


Section: header

        <header>
            <h1>Westringia Zena</h1>
            <h2><em>Westringia Fruticosa</em></h2>
        </header>

        <section id="quick-look">
            <p>The attractive Westringia Fruticosa ‘Zena’ is made more appealing by its hardiness, an attribute that typically characterises native Australian shrubs. Its ability to deal with salty winds makes it great for coastal plantings, hence, also making it suitable for coastal gardens.</p>
            <p>Being drought and frost-tolerant, basically hardy in many other locations, and possessing a sense of subtle charm and elegance, Westringia Zena is suitable for numerous garden types.</p>
            <p><strong>Synonyms:</strong> Coastal Rosemary, Native Rosemary, Australian Rosemary</p>
        </section>
    

Section: introduction

        <section id="introduction">
            <h2>Westringia Zena for a Charming and Resilient Garden</h2>
            <p>The Westringia Fruticosa ‘Zena’ is a hardy and compa

In [57]:
info_section_template = """
first example:
{
  "Typical height": "Up to 1 metre",
  "Minimum temperature tolerance": "Can tolerate temperatures around -2°C to 0°C",
  "Climate": "Suitable for coastal and dry inland areas",
  "Light conditions": "Full sun to part shade",
  "Growth habit": "Compact, bushy shrub",
  "Growth conditions": "Well-drained soil, also sandy or loamy"
}

another example:
{
  "Typical height": "4-6 metres",
  "Minimum temperature tolerance": "Can tolerate temperatures down to -5°C",
  "Climate": "Temperate to subtropical regions",
  "Light conditions": "Full sun to part shade",
  "Growth habit": "Compact, upright growth",
  "Growth conditions": "Well-drained soil, also slightly acidic to neutral soil"
}

another example:
{
  "Typical height": "Up to 6 metres",
  "Minimum temperature tolerance": "Tolerant of mild frost once established; prefers temperatures between 15-35°C but can tolerate brief periods of cold for as low as 10°C",
  "Climate": "Thrives in Australia's temperate climates",
  "Light conditions": "Full sun to partial shade; typically need 6 hours of full sun to properly grow",
  "Growth habit": "Dense, rounded shrub usually used for screening or hedging",
  "Growth conditions": "Needs well-drained, fertile soil, preferably clay, loamy or sandy soils"
}

"""

In [74]:
faq_section_template = """
<section id="faq">
    <h3><strong>Q: When is the best season to buy Magnolia Little Gem in Australia?</strong></h3>
    <p>The best time to buy Magnolia Little Gem is in autumn or early spring, so you can plant it when the weather is cooler and have it established before the harsh summer heat. It then bears its exquisite flowers in late summer and early autumn.</p>

    <h3><strong>Q: How much water does Magnolia Little Gem need?</strong></h3>
    <p>Water regularly after planting to establish the plant, then cut down the frequency as it becomes more drought-tolerant. During dry periods, water deeply but not often.</p>

    <h3><strong>Q: Can Magnolia Little Gem be grown in pots?</strong></h3>
    <p>Yes, thanks to its versatility, Magnolia Little Gem can be grown in large pots, which will look great in patios and small garden spaces.</p>

    <h3><strong>Q: How do I care for the Magnolia Little Gem in winter?</strong></h3>
    <p>Fortunately, Magnolia Little Gem is hardy and can withstand mild frost. If temperatures drop further, cover it with a frost cover. Mulch also protects roots from frost damage although it is expected for the tree to lose some leaves.</p>

    <h3><strong>Q: How big will Magnolia Little Gem get?</strong></h3>
    <p>It usually grows to a height of 4-6 metres, making it suitable for a variety of landscaping needs.</p>

    <h3><strong>Q: Can I buy Magnolia Little Gem in Brisbane?</strong></h3>
    <p>You can easily purchase Magnolia Little Gem online from our store, and we’ll deliver it directly to your Brisbane home.</p>
</section>
"""

In [30]:
def main_gpt_call(section_name, section_template, plant_name, keyword):
    prompt = f"""
    Generate SEO-optimized, creative HTML content for the section without any additional explanation, line breaks, or unnecessary information.
    Use the exact format as in the provided template for '{plant_name}', ensuring all content is highly relevant to the plant topic. If the keywords contain plants other than '{plant_name}', ignore them and focus only on {plant_name}. Make sure the content remains human-like and engaging, without sounding like a bot.

    Here’s an example for 'Coastal Rosemary' (Westringia Zena - Westringia Fruticosa):
    {section_template}
    
    Ensure tone and style are consistent with the example. Only return the HTML section; nothing more, nothing less.
    Keywords to use: {keyword}.
    """
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a creative content generator specializing in plant care and SEO optimization."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [63]:
def info_section_gpt_call(info_section_examples, plant_name):
    prompt = f"""
    Generate json formatted content for the section without any additional explanation, line breaks, or unnecessary information.
    Use the exact format as in the provided template for '{plant_name}', ensuring all content is highly relevant to the plant, and keep it concise, only 1 liner.
    Make sure the content remains human-like and engaging, without sounding like a bot. Make sure the tone and formatting is just like the one from the examples below.
    And don't change any units, keep it like in the example
    {info_section_examples}
    """
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a creative content generator specializing in plant care and SEO optimization."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [76]:
def faq_section_gpt_call(section_template, plant_name):
    prompt = f"""
    Generate SEO-optimized, creative HTML content in the form of FAQs without any additional explanation, line breaks, or unnecessary information.
    Use the exact format as in the provided template for '{plant_name}', ensuring all content is highly relevant to the plant topic. Focus entirely on FAQs related to {plant_name}, ensuring the content remains human-like and engaging, without sounding like a bot.
    Don't add more than 4 or 5 Question and Answers
    Here’s an example for 'Coastal Rosemary' (Westringia Zena - Westringia Fruticosa):
    {section_template}
    
    Ensure the tone and style are consistent with the example. Only return the HTML section; nothing more, nothing less.
    """
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a creative content generator specializing in plant care and SEO optimization."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [31]:
keywords = ", ".join(selected_keywords)

In [59]:
target_plant_name = "Pittosporum Tobira - Miss Muffet"

In [32]:
# Iterate through each section in the plant template
for section_name, section_html in plant_template_2.items():
    content = main_gpt_call(section_name, section_html, "Pittosporum Tobira - Miss Muffet", keywords)
    print(f"Section: {section_name}\n{content}\n")

Section: header
<header>
    <h1>Pittosporum Tobira - Miss Muffet</h1>
    <h2><em>Azarero Pittosporum Tobira</em></h2>
</header>

<section id="quick-look">
    <p>Pittosporum Tobira ‘Miss Muffet’ is a charming, hardy, and versatile plant, well-known as a compact and bushy evergreen, making it an ideal choice for various garden types and planting schemes. It's also commonly referred to as the Pitosporo Tobira, or simply Pitto Tobira, and is a variety of mock orange tree hailed for its delightful scented flowers.</p>
    <p>Often used for hedges (haie de pittosporum tobira) because of its impressive growth rate, Miss Muffet can reach a substantial height and requires only minimal pruning. Despite its robust growth, Pittosporum Miss Muffet's stature remains neat and compact, especially when allowed to grow in pots (pittosporum tobira en pot).</p>
    <p>Being quite hardy, it can withstand a variety of climates and conditions. Its small stature makes it suitable as a low hedge, border, or

In [64]:
info_section_response = info_section_gpt_call(info_section_template, target_plant_name)

In [80]:
print(info_section_response)

{
    "Typical height": "Up to 0.5 metres",
    "Minimum temperature tolerance": "Can tolerate temperatures down to -5°C",
    "Climate": "Ideally suited for Mediterranean climates",
    "Light conditions": "Thrives in full sun to semi-shade conditions",
    "Growth habit": "Compact, mounding shrub with attractive foliage",
    "Growth conditions": "Prefers well-drained soil, adaptable to clay, sandy or loamy soils"
}


In [67]:
def convert_to_json(output_string):
    try:
        # Using json.loads to convert the string to a dictionary (JSON object)
        json_object = json.loads(output_string)
        return json_object
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

In [68]:
# Convert to JSON
json_output = convert_to_json(info_section_response)

# Print the resulting JSON object
display(json_output)

{'Typical height': 'Up to 0.5 metres',
 'Minimum temperature tolerance': 'Can tolerate temperatures down to -5°C',
 'Climate': 'Ideally suited for Mediterranean climates',
 'Light conditions': 'Thrives in full sun to semi-shade conditions',
 'Growth habit': 'Compact, mounding shrub with attractive foliage',
 'Growth conditions': 'Prefers well-drained soil, adaptable to clay, sandy or loamy soils'}

In [69]:
type(json_output)

dict

In [77]:
faq_section_response = faq_section_gpt_call(faq_section_template, target_plant_name)

In [79]:
print(faq_section_response)

<section id="faq">
    <h3><strong>Q: When is the ideal time to plant Pittosporum Tobira - Miss Muffet in Australia?</strong></h3>
    <p>It is highly recommended to plant Pittosporum Tobira - Miss Muffet during autumn or early spring, to allow for the plant to establish prior to the summer heat. They bloom from late spring to early summer.</p>

    <h3><strong>Q: How often should I water my Pittosporum Tobira - Miss Muffet?</strong></h3>
    <p>Once established, Pittosporum Tobira - Miss Muffet requires moderate watering. In especially dry periods, ensure to water deeply but not frequently.</p>

    <h3><strong>Q: Can Pittosporum Tobira - Miss Muffet be pots?</strong></h3>
    <p>Indeed, Pittosporum Tobira - Miss Muffet is quite versatile and can thrive in large pots, making it suitable for elevated gardens and patio arrangements.</p>

    <h3><strong>Q: How do I take care of my Pittosporum Tobira - Miss Muffet in winter?</strong></h3>
    <p>Pittosporum Tobira - Miss Muffet is reason

In [70]:
def generate_html_from_dict(data_dict):
    html = '''
    <section id="detailed-information">
        <h2>Detailed Information</h2>
        <table border="1" cellpadding="10" cellspacing="0" style="width: 100%; border-collapse: collapse;">
    '''
    
    # Mapping dictionary keys to proper labels with capitalization
    for key, value in data_dict.items():
        # Convert dictionary key to title case
        formatted_key = key.replace('_', ' ').title()
        html += f'''
        <tr>
            <th style="text-align: left;">{formatted_key}</th>
            <td>{value}</td>
        </tr>
        '''
    
    html += '''
        </table>
    </section>
    '''
    
    return html

In [73]:
# Generate HTML from the dictionary
html_output = generate_html_from_dict(json_output)

# Print the generated HTML
print(html_output)


    <section id="detailed-information">
        <h2>Detailed Information</h2>
        <table border="1" cellpadding="10" cellspacing="0" style="width: 100%; border-collapse: collapse;">
    
        <tr>
            <th style="text-align: left;">Typical Height</th>
            <td>Up to 0.5 metres</td>
        </tr>
        
        <tr>
            <th style="text-align: left;">Minimum Temperature Tolerance</th>
            <td>Can tolerate temperatures down to -5°C</td>
        </tr>
        
        <tr>
            <th style="text-align: left;">Climate</th>
            <td>Ideally suited for Mediterranean climates</td>
        </tr>
        
        <tr>
            <th style="text-align: left;">Light Conditions</th>
            <td>Thrives in full sun to semi-shade conditions</td>
        </tr>
        
        <tr>
            <th style="text-align: left;">Growth Habit</th>
            <td>Compact, mounding shrub with attractive foliage</td>
        </tr>
        
        <tr>
 

In [33]:
def dummy_content():
    content= """
Section: header
<header>
    <h1>Pittosporum Tobira - Miss Muffet</h1>
    <h2><em>Azarero Pittosporum Tobira</em></h2>
</header>

<section id="quick-look">
    <p>Pittosporum Tobira ‘Miss Muffet’ is a charming, hardy, and versatile plant, well-known as a compact and bushy evergreen, making it an ideal choice for various garden types and planting schemes. It's also commonly referred to as the Pitosporo Tobira, or simply Pitto Tobira, and is a variety of mock orange tree hailed for its delightful scented flowers.</p>
    <p>Often used for hedges (haie de pittosporum tobira) because of its impressive growth rate, Miss Muffet can reach a substantial height and requires only minimal pruning. Despite its robust growth, Pittosporum Miss Muffet's stature remains neat and compact, especially when allowed to grow in pots (pittosporum tobira en pot).</p>
    <p>Being quite hardy, it can withstand a variety of climates and conditions. Its small stature makes it suitable as a low hedge, border, or even a standout specimen in various garden settings.</p>
    <p><strong>Synonyms:</strong> Azarero Pittosporum Tobira, Miss Muffet Pittosporum Tobira, Pitosporo Pittosporum Tobira, Pitosporo Tobira, Pitto Tobira, Le Pittosporum Tobira, Plante Pittosporum Tobira, Klebsame Pittosporum Tobira</p>
</section>

Section: introduction
<section id="introduction">
    <h2>Pittosporum Tobira 'Miss Muffet' for a Vibrant and Hardy Garden</h2>
    <p>The 'Miss Muffet' Pittosporum Tobira, also known as "pitosporo tobira", is a robust and compact plant that adapts well to different garden styles. Its moderate height makes it perfect for a low hedge or smaller gardens. If you are challenged by a limited garden space, this mock orange tree is an excellent choice.</p>
    <p>In terms of visual appeal, it features a thick green foliage and subtly fragrant flowers, which offer a delightful spectacle for the majority of the year. The plante pittosporum responds beautifully to trimming and shaping, making it an outstanding specimen for hedges and topiaries. This plant is also greatly sought after by gardeners for its slow but steady growth rate and its ability to tolerate drought, intervening minimally in its care requirements.</p>
    <p>Are you ready to enhance your garden with this robust plant known as pittosporum tobira 'nanum'? Here's an all-inclusive guide to help you get on the right track.</p>
</section>

Section: how-to-plant
<section id="how-to-plant">
    <h2>How to Plant Pittosporum Tobira 'Miss Muffet'?</h2>
    <p>For a remarkable touch of green in your garden, the Miss Muffet variety of Pittosporum Tobira is an excellent choice. Here's a guide to successfully planting and caring for your new Miss Muffet Pittosporum Tobira:</p>

    <ul>
        <li>Selecting the suitable location and soil type is essential when planting the azarero Pittosporum Tobira. It enjoys sunny or partially shaded areas and requires well-draining soil. The Pitosporo Tobira thrives best in sandy or loamy soils.</li>
        <li>Once you've picked the perfect spot, enhance your soil structure by incorporating compost or organic matter if necessary—they help improve drainage and soil nutrient content.</li>
        <li>After your Pittosporum mock orange tree is in the ground, provide a generous amount of water to help secure the soil around the roots.</li>
        <li>Keep in mind the Pittosporum Miss Muffet height when planting multiples—allow a distance of about 1 to 1.5 metres between each plant. This ensures proper air circulation and promotes optimal growth.</li>
        <li>Mulching around the base of your Pittosporum Tobira enano helps retain soil moisture, keeps weeds at bay, and aids in maintaining steady soil temperature.</li>
    </ul>
</section>

Section: why-part
<section id="why-part">
    <h2>Why Should You Buy Pittosporum Tobira 'Miss Muffet'?</h2>
    <p>The azarero Pitto Tobira, also known as the Pittosporum Tobira 'Miss Muffet', is a truly splendid plant. It delivers not only visual attraction but also numerous practical advantages, making it an ideal addition to your garden or outdoor space, some of the benefits we’ve highlighted for you below:</p>

    <ul>
        <li><strong>Scented Flowers:</strong> Pittosporum Tobira, the 'Mock Orange tree', blooms with fragrant, scented flowers, making your garden smell as beautiful as it looks.</li>
        <li><strong>Compact and Manageable Size:</strong> Known as Pittosporum Miss Muffet height, this plant grows to a perfect size for an average garden. Its growth rate and required pruning is conveniently low, ideal for those who appreciate a minimalist approach.</li>
        <li><strong>Easy Maintenance:</strong> The Miss Muffet Pittosporum Tobira is a low maintenance plant, perfect for those new to gardening or those with a busy lifestyle.</li>
        <li><strong>Resilient:</strong> This plant, also known as Pittosporum Tobira 'nanum' shows remarkable resilience. It’s an adaptable species that can tolerate various climates, making it well-suited for different environments globally.</li>
        <li><strong>Visual Appeal:</strong> Pitosporo Pittosporum Tobira is an aesthetically pleasing plant. It boasts lush green foliage that adds a great contrast to any garden space and enhances your outdoor décor.</li>
    </ul>
</section>

Section: planting_guide
<section id="planting-guide">
    <h2>How to Plant Pittosporum Tobira 'Miss Muffet'?</h2>
    <p>Miss Muffet Pittosporum Tobira is quite the head-turner in any garden, but even such a showstopper needs some TLC. After being planted, it is key to monitor the plant to catch pests or diseases early. Once established, minimal care is needed ensuring your 'Miss Muffet' stays vibrant.</p>

    <p>To ensure your Pittosporum Tobira stays at the height of its beauty, follow these simple steps:</p>

    <ul>
        <li>Regular watering is crucial during its initial growth phase, however, once established, only water during dry periods.</li>
        <li>Use a slow-release fertiliser once a year in spring to ensure your mock orange tree stays healthy.</li>
        <li>To encourage compact growth and a flush of fresh leaves, light pruning is recommended after flowering. Focus on the leggy growth if you notice any. <a href="your-pruning-guide-link" target="_blank">Find more pruning tips in our plant care guide.</a></li>
        <li>While the mock orange tree is hardy, it might still be damaged by frost, especially when young. Older, mature plants can handle light frost without trouble.</li>
    </ul>
</section>

Section: reasons_to_buy
<section id="reasons-to-buy">
    <h2>Why Should You Buy Pittosporum Tobira 'Miss Muffet'?</h2>
    <p>The Miss Muffet Pittosporum Tobira, or 'Azarero Pittosporum Tobira', as some like to call it, offers an abundance of charm both in visual and practical aspects. Here are some of the reasons:</p>

    <ul>

        <li><strong>Distinctive Foliage:</strong> The shiny, dark green leaves of Pittosporum Miss Muffet add depth to any garden, creating a visually appealing contrast to other flora.</li>
        <li><strong>Compact Size:</strong> Pittosporum Miss Muffet height makes it the ideal plant for small spaces. It grows to about a metre, perfect for borders, hedges, and screens.</li>
        <li><strong>Scented Flowers:</strong> Pittosporum Tobira’s white flowers are not only visually pleasing, they’re also fragrant, reminiscent of sweet citrus and mock orange tree blossoms.</li>
        <li><strong>Robust Growth Rate:</strong> With the right conditions, Pittosporum Tobira can grow rapidly and vigorously. As a hardy plant, it withstands a range of environmental conditions.</li>
        <li><strong>Low Maintenance:</strong> As long as basic needs like sun, water and soil are met, Pittosporum Tobira 'Miss Muffet' requires minimal pruning, perfect for new and busy gardeners alike.</li>
    </ul>
</section>

Section: care_guide
<section id="care-guide">
    <h2>How to Care for Pittosporum Tobira ‘Miss Muffet’?</h2>
    <p>Pittosporum Tobira 'Miss Muffet' is truly a resilient plant, however, it definitely benefits from a bit of care and attention, particularly when it is newly planted. Being vigilant on maintenance would help to notice any potential issues concerning diseases or pests.</p>
    <p>Follow these effortless steps to make your Miss Muffet Pittosporum Tobira flourish and to keep its pleasing outlook:</p>
    <ul>
        <li>Initially, make sure to water it frequently, but once it is established, water only during dry spells.</li>
        <li>We recommend fertilising once a year, preferably with a slow-release native plant fertiliser in the spring which will stimulate vigorous growth.</li>
        <li>After the flowering phase, light pruning will help to encourage denser and fresh growth and will maintain Pittosporum Miss Muffet's compact form. Prune back any excessive growth if needed. <a href="your-pruning-guide-link" target="_blank">Check out our in-depth guide on how to prune plants.</a></li>
        <li>The remarkable Pittosporum Tobira ‘Miss Muffet’ can withstand light frost and is remarkably hardy, but it might be necessary to offer protection to the juvenile plants in colder climate conditions. Adult plants will handle mild frost without problems.</li>
    </ul>
</section>

Section: growing_considerations
<section id="growing-considerations">
    <h2>Things to Consider When Growing Pittosporum Tobira - Miss Muffet</h2>
    <p>Developing the Pittosporum Tobira, often known as 'Pitosporo Tobira' or 'Pitto Tobira', most notably the 'Miss Muffet' variation, comes with its set of unique considerations that allow it to showcase its elegantly scented flowers.</p>

    <p>One key point to note is the varied growth rate of the Miss Muffet Pittosporum Tobira. Azarero Pittosporum Tobira can reach its maximum height swiftly under ideal conditions. However, the Tobira Nanum, a dwarf variety, grows at a slower pace. Therefore, understanding the growth pattern and height is key to successful gardening.</p>
    
    <p>However, one aspect that isn't time-consuming is the plant's maintenance. Miss Muffet Pittosporum Tobira can be kept compact through consistent pruning, particularly the 'Pittosporum Tobira Nanum Pruning'. This assists in maintaining the plant's attractive structure, whilst also preventing it from overshadowing other garden plants. Le Pittosporum Tobira, similar to any other tree, may develop root rot when subjected to poor drainage conditions‌. Making sure your Tobira Thunb Wt Aiton is placed in an area with good water drainage is essential.</p>

    <p>Deemed as the mock orange tree due to its heady citrus aroma, Pittosporum Scented Flowers are a delightful sight and scent. While its blossoms might not be as ostentatious as other flowering plants, the sweet fragrance more than makes up for it. Hence, if you’re looking for a beautiful and aromatic embellishment for your garden, Pianta Pittosporum Tobira might be the perfect choice.</p>
</section>

Section: pests_diseases
<section id="pests-diseases">
    <h2>Protecting Pittosporum Tobira - Miss Muffet from Pests and Diseases</h2>
    <p>Featuring sweetly scented flowers and robust growth rate, Pittosporum Tobira - Miss Muffet, also known as Pitto Tobira or Azarero Pittosporum Tobira, is a resilient plant, resisting many pests and diseases. But, insects such as scale and aphids can cause it discomfort. To combat these pests, garden enthusiasts often use safe treatments like neem oil or insecticidal soap.</p> 

    <p>Seen as a robust variety of the Pittosporum family, Miss Muffet can tolerate varying conditions but can be susceptible to fungal diseases if left in poorly drained soil. Keep your Pittosporum Tobira enano healthy by ensuring good drainage and avoiding overwatering. Are you unsure whether you're giving your plant too much or too little water? You can <a href="https://localbotanist.com.au/news/deciphering-plant-signals-the-telltale-signs-of-overwatering-vs-underwatering/">refer to this guide for assistance.</a></p>
</section>

Section: purchase
<section id="purchase">
    <h2>Where Can I Buy Pittosporum Tobira 'Miss Muffet'?</h2>
    <p>Looking to enhance your garden with the remarkable Pittosporum Tobira 'Miss Muffet'? Search no further. Here at the Local Botanist, you can find Miss Muffet Pittosporum Tobira, a plant with scented flowers that can reach a moderate height, offering your garden a unique aesthetic. Check our wide selection of plants and <a href="https://localbotanist.com.au/design-your-garden/">create your dream garden</a>.</p>

    <p>Do you want a garden style that matches the essence of Pittosporum Tobira 'Miss Muffet'? Try our <a href="https://localbotanist.com.au/quiz/">Garden Style Quiz</a> today.</p>
</section>

Section: companion_plants
<section id="companion-plants">
    <h2>Companion Plants for Pittosporum Tobira 'Miss Muffet'</h2>
    <p>Pittosporum Tobira Miss Muffet, also known as Azarero Pittosporum Tobira, is an adaptable, compact specimen complemented wonderfully by other low-maintenance and highly resilient plant species. Here are some excellent companions:</p>

    <ul>
        <li><strong>Pitosporo Pittosporum Tobira:</strong> This variant of Pittosporum Tobira offers complementing aesthetics, including a similar growth rate and height as the Miss Muffet.</li>
        <li><strong>Pittosporum Tenuifolium Nanum:</strong> This dwarf Pittosporum shares similar needs but adds a silvery hue with its gray-green leaves; both, when pruned, form a nice compact border.</li>
        <li><strong>Pittosporum Tobira Enano:</strong> A compact species native to Japan, it produces beautiful scented flowers harmonizing well with Miss Muffet's simple elegance.</li>
        <li><strong>Pitto Tobira:</strong> This variant shares similar drought-tolerance and growth habit, augmenting the visual appeal when planted alongside Miss Muffet.</li>
    </ul>
</section>
    """
    
    return content

In [34]:
def extract_sections(content):
    # Extract each section using regular expressions
    sections = re.findall(r'Section: (.*?)\n(.*?)(?=\nSection: |\Z)', content, re.DOTALL)
    return {name: html.strip() for name, html in sections}

In [35]:
# Get the dummy content
content = dummy_content()

# Extract sections
sections_dict = extract_sections(content)

In [36]:
# Create a DataFrame with the plant name and each section's content
plant_name = "Pittosporum Tobira 'Miss Muffet'"
data = {**{"plant name": plant_name}, **sections_dict}
df = pd.DataFrame([data])

In [37]:
df

,plant name,header,introduction,how-to-plant,why-part,planting_guide,reasons_to_buy,care_guide,growing_considerations,pests_diseases,purchase,companion_plants
0,Pittosporum Tobira 'Miss Muffet',<header>\n <h1>Pittosporum Tobira - Miss Mu...,"<section id=""introduction"">\n <h2>Pittospor...","<section id=""how-to-plant"">\n <h2>How to Pl...","<section id=""why-part"">\n <h2>Why Should Yo...","<section id=""planting-guide"">\n <h2>How to ...","<section id=""reasons-to-buy"">\n <h2>Why Sho...","<section id=""care-guide"">\n <h2>How to Care...","<section id=""growing-considerations"">\n <h2...","<section id=""pests-diseases"">\n <h2>Protect...","<section id=""purchase"">\n <h2>Where Can I B...","<section id=""companion-plants"">\n <h2>Compa..."


In [38]:
df.to_csv('Pittosporum Tobira.csv', index=False)

In [39]:
def create_html_for_row(df, row_number, output_dir="html_pages"):
    """
    Generates an HTML file for a specific row in the DataFrame.
    
    Parameters:
    - df: pandas DataFrame, should contain columns: 
        ['plant name', 'header', 'introduction', 'planting_guide', 'care_guide', 
        'reasons_to_buy', 'growing_considerations', 'pests_diseases', 'purchase', 
        'companion_plants']
    - row_number: the row index for which to generate the HTML file.
    - output_dir: directory to save the generated HTML file (default is "html_pages")
    
    Output:
    - Generates a .html file in the specified output directory.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Check if the specified row_number is valid
    if row_number < 0 or row_number >= len(df):
        print("Invalid row number. Please provide a valid row index.")
        return
    
    # Get the specified row
    row = df.iloc[row_number]
    
    # Get the plant name and use it to create a file name
    plant_name = row['plant name'].replace(" ", "_").lower()
    file_name = f"{plant_name}.html"
    file_path = os.path.join(output_dir, file_name)
    
    # Construct the HTML content
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{row['plant name']}</title>
</head>
<body>
    {row['header']}
    {row['introduction']}
    {row['planting_guide']}
    {row['care_guide']}
    {row['reasons_to_buy']}
    {row['growing_considerations']}
    {row['pests_diseases']}
    {row['purchase']}
    {row['companion_plants']}
</body>
</html>"""

    # Write the HTML content to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html_content)

    print(f"Generated {file_path}")

In [40]:
create_html_for_row(df, 0)

Generated html_pages\pittosporum_tobira_'miss_muffet'.html
